# Explaining Integer Linear Programming (ILP)

This is a very short guide that shows how [*Integer Linear Programming*](https://en.wikipedia.org/wiki/Integer_linear_programming) works. We will use the [Gurobi Optimizer](https://www.gurobi.com/) using Python for this (see this [example](ilp.ipynb) for details on how to set this up).

<!-- https://www.gurobi.com/documentation/9.0/refman/py_python_api_details.html -->

In [1]:
import gurobipy as gp
from gurobipy import GRB

We start by creating a model for the Gurobi optimizer.

In [ ]:
model = gp.Model();

## Variables

*Integer Linear Programming* is about finding assignments for integer variables that satisfy a set of constraints (of a particular form)—and that optionally are optimal with respect to some optimization statement.

Let's start with adding some variables to the model. We can add an *integer variable* $u_1$ as follows.

In [3]:
u1 = model.addVar(vtype=GRB.INTEGER, name="u1");

When we introduce a variable, we can also set a *lower bound* and an *upper bound* on the values that this variable is allowed to get. For example, let's add a variable whose value must be at least 0 and at most 5.

In [4]:
u2 = model.addVar(vtype=GRB.INTEGER, lb=0, ub=5, name="u2");

In the case where we have an integer variable with lower bound 0 and upper bound 1 (a *binary variable*), we can use the following notation:

In [5]:
u3 = model.addVar(vtype=GRB.BINARY, name="u3");

Often, it is convenient to add a sequence of variables. For example, suppose we want to add four binary variables. We can do this as follows.

In [6]:
v = model.addVars(4, vtype=GRB.BINARY, name="v");

This creates four variables that we can access as `v[0]`, `v[1]`, `v[2]` and `v[3]`.

## Linear constraints

Once we have added variables to the model, we can add constraints on the combinations of values that these variables are allowed to get. In *Integer Linear Programming*, we can only add so-called *linear constraints*. These are constraints of the form $c_1 * v_1 + \dotsm + c_n * v_n \leq d$, where $c_1,\dotsc,c_n$ and $d$ are constants, and $v_1,\dotsc,v_n$ are variables.

For example, we can add the constraint that $v_0 + 2*v_2 + 3*v_3 \leq 4$ as follows.

In [7]:
model.addConstr(v[0] + 2*v[2] + 3*v[3] <= 4);

If we combine several such linear constraints, we can also express constraints of the form $c_1 * v_1 + \dotsm + c_n * v_n = d$. We can do this by adding the constraints $c_1 * v_1 + \dotsm + c_n * v_n \leq d$ and $-c_1 * v_1 + \dotsm + -c_n * v_n \leq -d$. However, to make things easier, we can directly express this.

For example, we can add the constraint that $v_0+v_1+v_2 = 1$ as follows (`gp.quicksum()` expresses the sum of a list of variables).

In [8]:
model.addConstr(gp.quicksum([v[i] for i in [0,1,2]]) == 1);

## Optimization

We can also express an optimization statement: either maximizing or minimizing the value for some linear combination of variables.

For example, we can tell the solver to maximize the sum of $u_2$ and $u_3$ as follows.

In [9]:
model.setObjective(gp.quicksum([u2, u3]), GRB.MAXIMIZE);

## Finding an optimal model

Now that we have added variables, constraints and an optimization statement, we can tell the solver to find an assignment to the variables that satisfies all the lower and upper bounds on the variable values, the constraints that we added, and that is optimal with respect to the minimization/maximization statement that we added.

We do this as follows.

In [10]:
model.optimize();

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 7 columns and 6 nonzeros
Model fingerprint: 0x6389371b
Variable types: 0 continuous, 7 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 6.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%


The Gurobi optimizer gave us some statistics about the search process when we called `model.optimize()`.

If we want to access the values of the variables in the optimal model, we can do so as follows:

In [11]:
if model.status == GRB.OPTIMAL:
    for v in model.getVars():
        print("{}: {}".format(v.varName, v.x));
else:
    print("No optimal model found!");

u1: -0.0
u2: 5.0
u3: 1.0
v[0]: 1.0
v[1]: -0.0
v[2]: -0.0
v[3]: -0.0


The optimization statement is optional. We could also have left it out. If we don't add an optimization statement, the Gurobi optimizer will simply give us some assignment that satisfies all the constraints if we call `model.optimize()` (if such an assignment exists).

## Mixed-Integer Programming

Gurobi also allows us to add variables whose values are not restricted to integers (so-called *continuous variables*). The case where we combine integer variables and continuous variables is called *Mixed-Integer Programming (MIP)*.

For example, we can add two continuous variable $w_0,w_1$ with lower bound 0 and upper bound 2 as follows.

In [12]:
w = model.addVars(2, vtype=GRB.CONTINUOUS, lb=0, ub=2, name="w");

To see if these variables are really allowed to get non-integer values, let's add another constraint, and change the optimization statement to something else, and call `model.optimize()` again.

In [13]:
model.addConstr(7*w[0] + 3*w[1] == 2);

model.setObjective(gp.quicksum([w[i] for i in [0,1]]), GRB.MAXIMIZE);

model.optimize();

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3 rows, 9 columns and 8 nonzeros
Model fingerprint: 0x0bed1549
Variable types: 2 continuous, 7 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 4e+00]

MIP start from previous solve produced solution with objective 0.666667 (0.00s)
Loaded MIP start from previous solve with objective 0.666667

Presolve removed 3 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.666667 
No other solutions better than 0.666667

Optimal solution found (tolerance 1.00e-04)
Best objective 6.666666666667e-01, best bound 6.666666666667e-01, gap 0.0000%


If we now inspect the values of the variables in the assignment that the Gurobi optimizer gave us, we see that $w_1$ got a non-integer value.

In [14]:
if model.status == GRB.OPTIMAL:
    for v in model.getVars():
        print("{}: {}".format(v.varName, v.x));
else:
    print("No optimal model found!");

u1: -0.0
u2: -0.0
u3: 0.0
v[0]: 0.0
v[1]: 1.0
v[2]: 0.0
v[3]: 0.0
w[0]: 0.0
w[1]: 0.6666666666666666
